## Using Record to Model Immutable Data

* Java gives you several ways to create an immutable class
* most straightforward way is to create a final class with final fields and a constructor to initialize these fields
    - you need to add accessors for the fields
    - also need to add a toString(), equals(), and hashCode() that an IDE can generate for you
    - might also have to make this class serializable if you need to carry the instances from one app to another by sending over a network or file system
* so by creating a final class with final fields, the Point class may be a hundred lines long with code generated by your IDE just for an immutable aggregation of 2 integers
* so the best way to do this is through records
    - records can achieve all of the above with just a single line of code
    - just have to declare the state of a record and the rest is generated by the compiler

In [ ]:
// immutable class created with final fields and a constructor

public class Point {
    private final int x;
    private final int y;
    
    public Point(int x, int y) {
        this.x = x;
        this.y = y;
    }
}

## Calling Records to the Rescue

* the single line of code creates the following elements for you
    1. it is an immutable class with 2 fields: x and y, of type int
    2. has a canonical constructor to initialize these 2 fields
    3. toString(), equals(), and hashCode() methods have been created for you by the compilar with a default behavior that corresponds to what an IDE would have generated
        - can modify this behavior if you need by adding your own implementations of these methods
    4. can implement the Serializable interface so that you can send instances of Point to other applications over a network or through a file system
* reduces risk of bugs b/c everytime you modify the components of a record, the compiler automatically updates the equals() and hashCode() methods for you

In [ ]:
public record Point(int x, int y) {}

## The Class of a Record

* record is a class declared with the record keyword instead of the class keyword
* the class that the compiler creates for you when you create a record is final
* this class extends the java.lang.Record class
    - therefore, your record CANNOT extend any class
* a record can implement any number of interfaces

In [ ]:
public record Point(int x, int y) {}

## Declaring the Components of a Record

* the block that immediately follows the name of the record is (int x, int y)
    - this declares the components of the record named Point
    - for each component of a record, the compiler creates a private final field with the same name as this component
    - you can have any number of components declared in a record
    - essentially, int x and int y are components and private final fields are created for them
* the compiler also generates one accessor method for each component
    - the accessor method has the same name of the component and returns its value
* you are able to define your own accessor methods though
* the last elements generated by the compiler for you are overrides of the toString(), equals(), and hashCode() methods from the Object class

In [ ]:
// the 2 generated accessor methods

public int x() {
    return this.x;
}

public int y() {
    return this.y;
}

## Things You Cannot Add to a Record

* 3 things you cannot add to a record:
    1. cannot declare any instance field in a record
        - cannot add any instance field that would not correspond to a component
        - instance field = fields not declared with the static keyword
    2. cannot define any field initializer
    3. cannot add any instance initializer
* you can create static fields with initializers and static intializers

## Constructing a Record with its Cannonical Constructor

* compiler also creates a constructor for you called the canonical constructor
    - this constructor takes the components of your record as arguments and copies their values to the fields of the record class
* there are situations where you need to override this default behavior:
    1. you need to validate the state of your record
    2. you need to make a defensive copy of a mutable component

## Using the Compact Constructor

* you can use 2 different syntax to redefine the canonical constructor of a record
    - can use a compact constructor
    - or the canonical constructor itself
* the compact canonical constructor does not need to declare its block of parameters
    - with this syntax, you cannot directly assign the record's fields, e.g. this.start = start; since this is done for you by the compiler
    - but you can assign new values to the parameters which leads to the same result b/c the compiler-generated code will then assign these new values to the fields

In [ ]:
// suppose you have the following record

public record Range(int start, int end) {}

// for a record of this name, you would expect that the end is greater than the start
// thus you can add a validation rule by writing the compact constructor in your record

public record Range(int start, int end) {
    public Range {
        if (end <= start) {
            throw new IllegalArgumentException("End cannot be lesser than start");
        }
    }
}

In [ ]:
public Range {
    // set negative start and end to 0
    // by reassigning the compact constructor's
    // implicit parameters
    if (start < 0)
        start = 0;
    if (end < 0)
        end = 0;
}

## Using the Canonical Constructor

* in this case, the constructor you write needs to assign values to the fields of your record

In [ ]:
public record Range(int start, int end) {
    
    public Range(int start, int end) {
        if (end <= start) {
            throw new IllegalArgumentException("End cannot be lesser than start.");
        }
        if (start < 0) {
            this.start = 0;
        }
        else {
            this.start = start;
        }
        if (end > 100) {
            this.end = 10;
        }
        else {
            this.end = end;
        }
    }
}

## Defining any Constructor

* can also add any constructor to a record as long as this constructor calls the canonical constructor of your record
    - the call to this() must be the first statement of your constructor
* for the State record example below, it is defined on 3 components:
    1. the name of this state
    2. the name of the capital of this state
    3. a list of city names that may be empty
* we need to store a defensive copy of the list of cities to ensure that it will not be modified from outside of this record
    - a defensive copy is basically a different object that has the same values of its original
    - any modifications to the defensive copy or original are not observed in the other
* for the example:
    - having a constructor that does not take any city is useful in your app so this can be another constructor that only takes the state name and capital city name
        * this second constructor must call the canonical constructor
    - also, instead of passing a list of cities, you can pass the cities as a vararg (variable # of arguments) which you can do with a third constructor
        * this third constructor must also call the canonical constructor with the proper list
        * the vararg is basically the spread syntax that allows an arbitrary number of parameters to be passed in
            - it is built using arrays under the hood

In [ ]:
public record State(String name, String capitalCity, List<String> cities) {
    
    // canonical constructor
    public State {
        // List.copyOf returns an unmodifiable copy,
        // so the list assigned to `cities` can't change anymore
        cities = List.copyOf(cities);
    }
    
    // second constructor
    // does not take any city
    public State(String name, String capitalCity) {
    
        // canonical constructor called with this()
        // passes in the same parameters but with an empty List of cities
        
        // List.copyOf() does not accept null values in the collection it gets as an argument
        this(name, capitalCity, List.of());
    }
    
    // third constructor
    // passes in any number of cities as an argument, not as a list of cities
    // e.g. (California, Sacramento, San Francisco, Daly City, San Jose);
    public State(String name, String capitalCity, String... cities) {
        this(name, capitalCity, List.of(cities));
    }
    
    
}

## Getting the State of a Record

* don't need to add any accessor to a record
    - the compiler does that for you
    - the name of the accessor methods is the same as the component
        * e.g. int x will have accessor x() and int y will have accessor y()
* you can create your own accessors though

In [1]:
// assuming the accessor method of the State record did not create an immutable copy of the cities list
// you can define your own accessor that makes a defensive copy and returns it

public List<String> cities() {
    return List.copyOf(cities);
}

## Serializing Records

* records can be serialized/deserialized if your record class implements Serializable but there are restrictions:
    1. none of the systems you can use to replace the default serialization process are available for records
        - creating a writeObject() and readObject() method has no effect nor implementing Externalizable
    2. records can be used as proxy objects to serialize other objects
        - a readResolve() method can return a record
        - adding a writeReplace() in a record is also possible
    3. Deserializing a record _always_ calls the canonical constructor
        - so all the validation rules you may add in this constructor will be enforced when deserializing a record
* this makes records a very good choice for creating data transport objects in your app

## Using Records in a Real Use Case

* you can carry data in the object model of your application
    - you can use records as an immutable data carrier
* you can declare local records as well to improve readability of your code
* for example:
    - you have 2 entities modeled as records: City and State
    - you need to compute the state that has the greatest number of cities
    - can use the Stream API to build the histogram of the states with the number of cities each one has
    - this histogram is modled by a map
* for these 2 entities:
    - you can create a _local record_ that aggregates a state and the number of cities in this state
    - it has a constructor that takes an instance of Map.Entry as a parameter to map the stream of key-value pairs to a stream of records
    - because you need to compare these aggregates by the number of cities, you can add a factory method to provide this comparator

In [ ]:
// 2 entities: City and State
public record City(String name, State state) {}

public record State(String name) {}

In [ ]:
// building a histogram of # of cities per state
List<City> cities = List.of();

Map<State, Long> numberOfCitiesPerState = 
    cities.stream()
        .collect(Collectors.groupingBy(
            City::state, Collectors.counting()
        ));

In [ ]:
// getting max of this histogram
Map.Entry<State, Long> stateWithTheMostCities =
    numberOfCitiesPerState.entrySet().stream()
        .max(Map.Entry.comparingByValue())
        .orElseThrow();

In [ ]:
// can use a local record to aggregate state and number of cities
record NumberOfCitiesPerState(State state, long numberOfCities) {
    
    public NumberOfCitiesPerState(Map.Entry<State, Long> entry) {
        // constructor that calls the canonical constructor using this()
        // the compiler already created the state and numberOfCities fields
        // and all this code does is it retrieves the state and numberOfCities from the Map entry
        this(entry.getKey(), entry.getValue());
    }
    
    public static Comparator<NumberOfCitiesPerState> comparingByNumberOfCities() {
        return Comparator.comparing(NumberOfCitiesPerState::numberOfCities);
    }
}

NumberOfCitiesPerState stateWithTheMostCities = 
    numberOfCitiesPerState.entrySet().stream()
        .map(NumberOfCitiesPerState::new)
        .max(NumberOfCitiesPerState.comparingByNumberOfCities())
        .orElseThrow();